A notebook for visualizing movies of raw flouresence for datasets

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib qt

In [15]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd
import pyqtgraph as pg
import pyspark

from janelia_core.fileio.exp_reader import find_images
from janelia_core.dataprocessing.utils import get_processed_image_data

from keller_zlatic_vnc.data_processing import generate_standard_id_for_volume

In [56]:
# Location of excel file specifying where the data for each experiment is saved relative to the base folder
data_loc_file = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'

# Additional parameters (constant for all datasets) specifying where the data is stored
image_base_folder =r'K:\\SV4'
image_processed_folder = 'Results\\WeightFused'
image_ext = r'weightFused.TimeRegistration.templateSpace.klb'

# Extension for image files 
img_file_ext = 'weightFused.TimeRegistration.templateSpace.klb'

# Subject id for which we want to load data
s_id = 'CW_17-11-27-L4'

## Read in excel file specifying location of each dataset

In [6]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

volume_locs = pd.read_excel(data_loc_file, header=1, usecols=[1, 2], converters=converters)

## Get subject ids for each volume

In [7]:
volume_subjs = [generate_standard_id_for_volume(volume_locs.loc[i,'Main folder'], 
                                                       volume_locs.loc[i,'Subfolder'])  for i in volume_locs.index]

## Find images for the subject

In [57]:
d_ind = np.argwhere(np.asarray(volume_subjs) == s_id)[0][0]
data_main_folder = volume_locs['Main folder'][d_ind]
data_sub_folder = volume_locs['Subfolder'][d_ind]

In [58]:
base_data_dir = Path(image_base_folder) / data_main_folder / data_sub_folder / image_processed_folder

In [59]:
imgs = find_images(image_folder=base_data_dir, image_ext=img_file_ext, image_folder_depth=1)  

Searching for image files...
Found 6269 images.


## Create a spark context

In [ ]:
conf = pyspark.SparkConf().setMaster('local[20]').setAll([
    ('spark.executor.memory', '10g'), ('spark.driver.memory','400g'), ('spark.driver.maxResultSize', '300g')])
sc = pyspark.SparkContext(conf=conf)

## Pick some images to view and wisualize movie

In [63]:
img_inds = slice(115, 128)

In [64]:
read_imgs = np.stack(get_processed_image_data(imgs[img_inds], func=lambda x: np.max(x, axis=0), sc=sc))

In [65]:
pg.image(read_imgs)